In [49]:
import pymorphy2
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import ngrams
import re
import pandas as pd
import string
from nltk.corpus import stopwords
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
def standart_data(data):
  stop_words = set(stopwords.words('russian'))
  morph = pymorphy2.MorphAnalyzer()
  words = [re.sub(r'\d+', '', i.lower()).replace('»', '').replace('«', '').translate(str.maketrans('', '', string.punctuation)) for i in data.split()]
  quest = []
  for i in words:
    quest.append([morph.parse(w)[0].normal_form for w in nltk.word_tokenize(i)])# if w not in stop_words])
  quest = [sorted(list(set([j for j in i]))) for i in quest if i != []]
  return ' '.join([' '.join(i) for i in quest])

In [67]:
def n_grams(text, n):
    n_grams1 = ngrams(standart_data(text).split(), n)
    return [' '.join(grams) for grams in n_grams1]

In [70]:
def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

In [111]:
data_df = {'Comment': ['Товар оказался дорогим и также его не было на складе', 
                       'В магазине не продается данная модель телефона',
                       'Клиент решил отменить покупку после того, как ему позвонил сын и сказал, что уже купил',
                       'Покупатель искал стиральную машинку размером 345*432, но у нас нет машинки таких габаритов',
                       'В нашем магазине не продается алкоголь, который хотел купить посетитель', 
                       'Клиент сказал, что ему нужно подумать. Я поставил оповещение на 10 число',
                       'Мужчина искал спиннинг определенного размера, у нас такого не оказалось',
                       'Посетитель отменил покупку, так как не хватило денег, а кредит не одобрили',
                       'Женщина еще посоветуется с семьей, прежде чем купить телевизор. Напомню ей 25 июня',
                       'Мужина сказал, что пришал праста посмотреть на тавары, консультировал по смартфонам'
                       ], 
           'Tags': [[]] * 10, 
           'Employee_id': [i for i in range(1, 6)]* 2}
df = pd.DataFrame(data_df)
df['Comment_ngrams'] = [n_grams(standart_data(i), 6) for i in df['Comment']]

In [112]:
df

,Comment,Tags,Employee_id,Comment_ngrams
0,Товар оказался дорогим и также его не было на ...,[],1,"[товар оказаться дорога и также он, оказаться ..."
1,В магазине не продается данная модель телефона,[],2,"[в магазин не продаваться дать модель, магазин..."
2,"Клиент решил отменить покупку после того, как ...",[],3,"[клиент решить отменить покупка после тот, реш..."
3,Покупатель искал стиральную машинку размером 3...,[],4,[покупатель искать стиральный машинка размер н...
4,"В нашем магазине не продается алкоголь, которы...",[],5,"[в наш магазин не продаваться алкоголь, наш ма..."
5,"Клиент сказал, что ему нужно подумать. Я поста...",[],1,"[клиент сказать что он нужно подумать, сказать..."
6,"Мужчина искал спиннинг определенного размера, ...",[],2,[мужчина искать спиннинг определённый размер у...
7,"Посетитель отменил покупку, так как не хватило...",[],3,"[посетитель отменить покупка так как не, отмен..."
8,"Женщина еще посоветуется с семьей, прежде чем ...",[],4,"[женщина ещё посоветоваться с семья прежде, ещ..."
9,"Мужина сказал, что пришал праста посмотреть на...",[],5,"[мужин сказать что пришалый праста посмотреть,..."


In [120]:
key_words = {'Дорого': [['дорог'], ['не', 'денег'], ['не', 'деньг']], 
             'Нет в наличии': [['не', 'склад']], 
             'Передумал': ['отменить', 'передумать'], 
             'Стиральные машины': ['стираль'], 
             'Нет подходящего товара': [['не', 'прода']],
             'Нет подходящего размера': ['габарит', 'размер'],
             'Спиннинг': ['спиннинг'],
             'Нужно подумать': ['думать', 'подумать', 'совет'],
             'Оповещение': ['напомнить', 'помнить', 'оповещение', 'напоминание'],
             'Просмотр': ['посмотреть', 'ознакомиться'],
             'Телевизор': ['телевизор'],
             'Кредит': ['кредит'],
             'Телефон': ['телефон', 'смартфон']
             }

In [121]:
global_tags = []
for k in range(len(df)):
  list_tags = []
  for v in range(len(df['Comment_ngrams'][k])):
    for i in key_words.values():
      for j in i:
        if type(j) == list and len(j) > 1:
          if j[0] in df['Comment_ngrams'][k][v] and j[1] in df['Comment_ngrams'][k][v]:
            if get_key(key_words, i) not in list_tags:
              list_tags += [get_key(key_words, i)]
        elif type(j) == list and len(j) == 1:
          if j[0] in df['Comment_ngrams'][k][v]:
            if get_key(key_words, i) not in list_tags:
              list_tags += [get_key(key_words, i)]
        else:
          if j in df['Comment_ngrams'][k][v]:
            if get_key(key_words, i) not in list_tags:
              list_tags += [get_key(key_words, i)]
  global_tags.append(list_tags)

In [125]:
df['Tags'] = global_tags

In [126]:
df

,Comment,Tags,Employee_id,Comment_ngrams
0,Товар оказался дорогим и также его не было на ...,"[Дорого, Нет в наличии]",1,"[товар оказаться дорога и также он, оказаться ..."
1,В магазине не продается данная модель телефона,"[Нет подходящего товара, Телефон]",2,"[в магазин не продаваться дать модель, магазин..."
2,"Клиент решил отменить покупку после того, как ...",[Передумал],3,"[клиент решить отменить покупка после тот, реш..."
3,Покупатель искал стиральную машинку размером 3...,"[Стиральные машины, Нет подходящего размера]",4,[покупатель искать стиральный машинка размер н...
4,"В нашем магазине не продается алкоголь, которы...",[Нет подходящего товара],5,"[в наш магазин не продаваться алкоголь, наш ма..."
5,"Клиент сказал, что ему нужно подумать. Я поста...","[Нужно подумать, Оповещение]",1,"[клиент сказать что он нужно подумать, сказать..."
6,"Мужчина искал спиннинг определенного размера, ...","[Нет подходящего размера, Спиннинг]",2,[мужчина искать спиннинг определённый размер у...
7,"Посетитель отменил покупку, так как не хватило...","[Передумал, Дорого, Кредит]",3,"[посетитель отменить покупка так как не, отмен..."
8,"Женщина еще посоветуется с семьей, прежде чем ...","[Нужно подумать, Телевизор, Оповещение]",4,"[женщина ещё посоветоваться с семья прежде, ещ..."
9,"Мужина сказал, что пришал праста посмотреть на...","[Просмотр, Телефон]",5,"[мужин сказать что пришалый праста посмотреть,..."
